# **5장 단순 최저 운송사 선정**

- 문제1: 단순한 제약조건을 고려하는 최저 운송사 선정 문제
- 2개의 함수로 구성된 전체 코드와 세부 기능을 확인하는 상세 코드로 구성되어 있음
- 코드생성 프롬프트에 코드에 대한 주석처리 요청 추가
- 수리모델을 풀어쓰기 함수 생성 추가

In [ ]:
pip install pulp

In [ ]:
# 필요한 라이브러리 불러오기
import pandas as pd  # 엑셀 데이터를 다루기 위한 라이브러리
import time  # 수행 시간 측정용
import pulp  # 선형계획법 최적화 라이브러리
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpInteger, value  # PuLP에서 자주 쓰는 클래스 및 함수들

# 최적화 함수 정의
def optimize_simple_assignment():
    start_time = time.time()  # 시작 시간 기록

    #--------------------------------------------------------
    # 1) 엑셀에서 데이터 읽어오기
    #--------------------------------------------------------
    input_file = "/content/sample_data/단순최적화_기준정보.xlsx"  # 입력 엑셀 파일 경로

    # 운송사 관련 데이터 읽기 (A~E열: 운송사, 운송구간, 트럭유형, 운송단가, 트럭대수)
    df = pd.read_excel(input_file, sheet_name=0, usecols="A:E")

    # 수요 데이터 읽기 (H~J열: 운송구간, 트럭유형, 필요대수)
    df_demand = pd.read_excel(input_file, sheet_name=0, usecols="H:J")
    df_demand.columns = ['운송구간', '트럭유형', '필요대수']  # 컬럼명 재정의

    # 데이터 타입 정리 및 결측치 제거
    df['운송단가'] = pd.to_numeric(df['운송단가'], errors='coerce')
    df['트럭대수'] = pd.to_numeric(df['트럭대수'], errors='coerce')
    df_demand['필요대수'] = pd.to_numeric(df_demand['필요대수'], errors='coerce')
    df.dropna(subset=['운송단가', '트럭대수'], inplace=True)
    df_demand.dropna(subset=['필요대수'], inplace=True)

    # 무한대 값(Inf, -Inf)이 있는 데이터도 제거
    df = df[(~df['운송단가'].isin([float('inf'), float('-inf')])) &
        (~df['트럭대수'].isin([float('inf'), float('-inf')]))]
    df_demand = df_demand[(~df_demand['필요대수'].isin([float('inf'), float('-inf')]))]

    #--------------------------------------------------------
    # 2) 파라미터 구성 (딕셔너리 형태로 정리)
    #--------------------------------------------------------
    cost_dict = {}      # 운송단가: {(운송사, 구간, 트럭유형): 단가}
    capacity_dict = {}  # 운송사별 공급 가능한 트럭 수: {(운송사, 구간, 트럭유형): 대수}
    # 데이터프레임 df에서 각 행(row)을 읽어와 딕셔너리에 저장합니다.
    for _, row in df.iterrows():
        key = (row['운송사'], row['운송구간'], row['트럭유형'])  # 딕셔너리의 키로 사용할 튜플
        cost_dict[key] = row['운송단가']  # 운송단가 저장
        capacity_dict[key] = row['트럭대수']  # 트럭대수 저장

    demand_dict = {}  # 수요 트럭 수: {(구간, 트럭유형): 필요대수}
    for _, row in df_demand.iterrows():
        demand_dict[(row['운송구간'], row['트럭유형'])] = row['필요대수']

    truckers = df['운송사'].unique()  # 운송사 목록 추출
    route_type_list = demand_dict.keys()  # 수요 조건에 해당하는 (구간, 트럭유형) 조합

    #--------------------------------------------------------
    # 3) PuLP 최적화 모델 생성
    #--------------------------------------------------------
    prob = LpProblem("Simple_Optimization", LpMinimize)  # 목적: 비용 최소화

    # 결정변수 정의: x_(운송사)_(구간)_(트럭유형) = 몇 대 배정할지
    x_vars = {}
    for (i, r, t), cost in cost_dict.items():
        x_vars[(i, r, t)] = LpVariable(f"x_{i}_{r}_{t}", lowBound=0, cat=LpInteger)  # 정수변수로 정의

    #--------------------------------------------------------
    # 4) 목적함수 정의: 전체 운송비용 최소화
    #--------------------------------------------------------
    prob += lpSum([
        cost_dict[(i, r, t)] * x_vars[(i, r, t)]
        for (i, r, t) in cost_dict
    ]), "Minimize_TotalCost"

    #--------------------------------------------------------
    # 5) 제약조건 정의
    #--------------------------------------------------------

    # 제약조건 1: 각 (구간, 트럭유형)에 대해 수요만큼 트럭이 배정되어야 함
    for (r, t) in route_type_list:
        prob += lpSum([
            x_vars[(i, r, t)]
            for i in truckers if (i, r, t) in x_vars
        ]) == demand_dict[(r, t)], f"Demand_{r}_{t}"

    # 제약조건 2: 운송사별 보유 트럭 수를 초과하지 않도록
    for (i, r, t), cap in capacity_dict.items():
        prob += x_vars[(i, r, t)] <= cap, f"Capacity_{i}_{r}_{t}"

    #--------------------------------------------------------
    # 6) 최적화 실행
    #--------------------------------------------------------
    prob.solve(pulp.PULP_CBC_CMD(msg=0))  # CBC Solver를 사용하여 최적화 수행

    #--------------------------------------------------------
    # 7) 결과 출력
    #--------------------------------------------------------
    print("Status:", pulp.LpStatus[prob.status])  # 최적화 상태 출력
    if prob.status not in [1, 2]:  # Optimal(1) 또는 Feasible(2)인 경우만 유효
        print("=> 유효한 해를 찾지 못했습니다.")
        return None

    # 총 운송비 계산
    total_cost = value(prob.objective)
    print("총 운송비:", total_cost)

    # 최적화 결과(트럭 배정 현황) 정리
    result_data = []
    for (i, r, t), var in x_vars.items():
        if var.varValue > 0:  # 배정된 트럭 수가 0보다 큰 경우만 출력
            result_data.append([i, r, t, cost_dict[(i, r, t)], int(var.varValue)])

    # 결과를 데이터프레임으로 출력
    df_result = pd.DataFrame(result_data, columns=['운송사', '운송구간', '트럭유형', '운송단가', '선택된트럭수'])
    print("\n--- 배정 결과 ---")
    print(df_result.to_string(index=False))  # 결과 출력

    # 엑셀로 저장
    output_file = "/content/sample_data/단순최적화결과.xlsx"
    df_result.to_excel(output_file, index=False)

    end_time = time.time()  # 종료 시간 기록
    print("\n프로그램 수행시간(초):", round(end_time - start_time, 4))  # 수행 시간 출력

    return prob  # 최적화 모델 객체 반환

# 최적화 모델 상세 출력 함수
def display_optimization_model(prob):
    """
    최적화 모델의 목적식과 제약식을 출력하는 함수
    """
    print("\n=== 목적함수 ===")
    print(prob.objective)  # 목적함수 출력

    print("\n=== 제약조건 ===")
    for name, constraint in prob.constraints.items():  # 제약조건 모두 출력
        print(f"{name}: {constraint}")

    print("\n=== 결정변수 ===")
    for var in prob.variables():  # 변수명과 해 출력
        print(f"{var.name}: {var.varValue}")

# 메인 실행부
if __name__ == "__main__":
    # 최적화 실행
    prob = optimize_simple_assignment()

    # 모델 내용 출력
    if prob is not None:
        display_optimization_model(prob)


Status: Optimal
총 운송비: 1300.0

--- 배정 결과 ---
  운송사 운송구간 트럭유형  운송단가  선택된트럭수
운송사01 구간01 트럭01   100       2
운송사01 구간01 트럭02   100       2
운송사01 구간02 트럭01   100       2
운송사02 구간02 트럭02   350       2

프로그램 수행시간(초): 0.1639

=== 목적함수 ===
100*x_운송사01_구간01_트럭01 + 100*x_운송사01_구간01_트럭02 + 100*x_운송사01_구간02_트럭01 + 700*x_운송사01_구간02_트럭02 + 150*x_운송사02_구간01_트럭01 + 150*x_운송사02_구간01_트럭02 + 150*x_운송사02_구간02_트럭01 + 350*x_운송사02_구간02_트럭02 + 200*x_운송사03_구간01_트럭01 + 200*x_운송사03_구간01_트럭02 + 200*x_운송사03_구간02_트럭01 + 400*x_운송사03_구간02_트럭02

=== 제약조건 ===
Demand_구간01_트럭01: x_운송사01_구간01_트럭01 + x_운송사02_구간01_트럭01 + x_운송사03_구간01_트럭01 = 2.0
Demand_구간01_트럭02: x_운송사01_구간01_트럭02 + x_운송사02_구간01_트럭02 + x_운송사03_구간01_트럭02 = 2.0
Demand_구간02_트럭01: x_운송사01_구간02_트럭01 + x_운송사02_구간02_트럭01 + x_운송사03_구간02_트럭01 = 2.0
Demand_구간02_트럭02: x_운송사01_구간02_트럭02 + x_운송사02_구간02_트럭02 + x_운송사03_구간02_트럭02 = 2.0
Capacity_운송사01_구간01_트럭01: x_운송사01_구간01_트럭01 <= 2
Capacity_운송사02_구간01_트럭01: x_운송사02_구간01_트럭01 <= 2
Capacity_운송사03_구간01_트럭01: x_운송사03_구간01_트럭01

# 최적함 함수 정의
def optimize_simple_assignment():

In [ ]:
start_time = time.time()  # 시작 시간 기록

# 기준정보의 운송단가와 공급제약 데이터 읽기

In [ ]:
    input_file = "/content/sample_data/단순최적화_기준정보.xlsx"  # 입력 엑셀 파일 경로
    # 운송사 관련 데이터 읽기 (A~E열: 운송사, 운송구간, 트럭유형, 운송단가, 트럭대수)
    df = pd.read_excel(input_file, sheet_name=0, usecols="A:E")

In [ ]:
df

,운송사,운송구간,트럭유형,운송단가,트럭대수
0,운송사01,구간01,트럭01,100,2
1,운송사02,구간01,트럭01,150,2
2,운송사03,구간01,트럭01,200,2
3,운송사01,구간01,트럭02,100,2
4,운송사02,구간01,트럭02,150,2
5,운송사03,구간01,트럭02,200,2
6,운송사01,구간02,트럭01,100,2
7,운송사02,구간02,트럭01,150,2
8,운송사03,구간02,트럭01,200,2
9,운송사01,구간02,트럭02,700,2


#도표 5-24. 기준정보의 수요 데이터 읽기

In [ ]:
    # 수요 데이터 읽기 (H~J열: 운송구간, 트럭유형, 필요대수)
    df_demand = pd.read_excel(input_file, sheet_name=0, usecols="H:J")
    df_demand.columns = ['운송구간', '트럭유형', '필요대수']  # 컬럼명 재정의

In [ ]:
df_demand

,운송구간,트럭유형,필요대수
0,구간01,트럭01,2.0
1,구간01,트럭02,2.0
2,구간02,트럭01,2.0
3,구간02,트럭02,2.0
4,NaN,NaN,NaN
5,NaN,NaN,NaN
6,NaN,NaN,NaN
7,NaN,NaN,NaN
8,NaN,NaN,NaN
9,NaN,NaN,NaN


# 도표 5-25. 결측치 제거

In [ ]:
    # 데이터 타입 정리 및 결측치 제거
    df['운송단가'] = pd.to_numeric(df['운송단가'], errors='coerce')
    df['트럭대수'] = pd.to_numeric(df['트럭대수'], errors='coerce')
    df_demand['필요대수'] = pd.to_numeric(df_demand['필요대수'], errors='coerce')
    df.dropna(subset=['운송단가', '트럭대수'], inplace=True)
    df_demand.dropna(subset=['필요대수'], inplace=True)

In [ ]:
df_demand

,운송구간,트럭유형,필요대수
0,구간01,트럭01,2.0
1,구간01,트럭02,2.0
2,구간02,트럭01,2.0
3,구간02,트럭02,2.0


# 도표 5-28. 무한대 값 제거 파이썬 코드와 예시

In [ ]:
# 무한대 값(Inf, -Inf)이 있는 데이터도 제거합니다.
df = df[(~df['운송단가'].isin([float('inf'), float('-inf')])) &
        (~df['트럭대수'].isin([float('inf'), float('-inf')]))]

df_demand = df_demand[(~df_demand['필요대수'].isin([float('inf'), float('-inf')]))]

In [ ]:
df

,운송사,운송구간,트럭유형,운송단가,트럭대수
0,운송사01,구간01,트럭01,100,2
1,운송사02,구간01,트럭01,150,2
2,운송사03,구간01,트럭01,200,2
3,운송사01,구간01,트럭02,100,2
4,운송사02,구간01,트럭02,150,2
5,운송사03,구간01,트럭02,200,2
6,운송사01,구간02,트럭01,100,2
7,운송사02,구간02,트럭01,150,2
8,운송사03,구간02,트럭01,200,2
9,운송사01,구간02,트럭02,700,2


# 딕셔너리 저장

In [ ]:
    cost_dict = {}      # 운송단가: {(운송사, 구간, 트럭유형): 단가}
    capacity_dict = {}  # 운송사별 공급 가능한 트럭 수: {(운송사, 구간, 트럭유형): 대수}
    # 데이터프레임 df에서 각 행(row)을 읽어와 딕셔너리에 저장합니다.
    for _, row in df.iterrows():
        key = (row['운송사'], row['운송구간'], row['트럭유형'])  # 딕셔너리의 키로 사용할 튜플
        cost_dict[key] = row['운송단가']  # 운송단가 저장
        capacity_dict[key] = row['트럭대수']  # 트럭대수 저장

    print(cost_dict)
    print(capacity_dict)

{('운송사01', '구간01', '트럭01'): 100, ('운송사02', '구간01', '트럭01'): 150, ('운송사03', '구간01', '트럭01'): 200, ('운송사01', '구간01', '트럭02'): 100, ('운송사02', '구간01', '트럭02'): 150, ('운송사03', '구간01', '트럭02'): 200, ('운송사01', '구간02', '트럭01'): 100, ('운송사02', '구간02', '트럭01'): 150, ('운송사03', '구간02', '트럭01'): 200, ('운송사01', '구간02', '트럭02'): 700, ('운송사02', '구간02', '트럭02'): 350, ('운송사03', '구간02', '트럭02'): 400}
{('운송사01', '구간01', '트럭01'): 2, ('운송사02', '구간01', '트럭01'): 2, ('운송사03', '구간01', '트럭01'): 2, ('운송사01', '구간01', '트럭02'): 2, ('운송사02', '구간01', '트럭02'): 2, ('운송사03', '구간01', '트럭02'): 2, ('운송사01', '구간02', '트럭01'): 2, ('운송사02', '구간02', '트럭01'): 2, ('운송사03', '구간02', '트럭01'): 2, ('운송사01', '구간02', '트럭02'): 2, ('운송사02', '구간02', '트럭02'): 2, ('운송사03', '구간02', '트럭02'): 2}


In [ ]:
    demand_dict = {}  # 수요 트럭 수: {(구간, 트럭유형): 필요대수}
    for _, row in df_demand.iterrows():
        demand_dict[(row['운송구간'], row['트럭유형'])] = row['필요대수']

    truckers = df['운송사'].unique()  # 운송사 목록 추출
    route_type_list = demand_dict.keys()  # 수요 조건에 해당하는 (구간, 트럭유형) 조합

In [ ]:
truckers, route_type_list

(array(['운송사01', '운송사02', '운송사03'], dtype=object),
 dict_keys([('구간01', '트럭01'), ('구간01', '트럭02'), ('구간02', '트럭01'), ('구간02', '트럭02')]))

# 도표 5-32. 최적화 모델의 객체 생성

In [ ]:
prob = LpProblem("Simple_Optimization", LpMinimize)  # 목적: 비용 최소화
print(prob)

Simple_Optimization:
MINIMIZE
None
VARIABLES



# 도표 5-33. 결정변수의 정의와 저장

In [ ]:
    # 결정변수 정의: x_(운송사)_(구간)_(트럭유형) = 몇 대 배정할지
    x_vars = {}
    for (i, r, t), cost in cost_dict.items():
        x_vars[(i, r, t)] = LpVariable(f"x_{i}_{r}_{t}", lowBound=0, cat=LpInteger)  # 정수변수로 정의
    print(x_vars)

{('운송사01', '구간01', '트럭01'): x_운송사01_구간01_트럭01, ('운송사02', '구간01', '트럭01'): x_운송사02_구간01_트럭01, ('운송사03', '구간01', '트럭01'): x_운송사03_구간01_트럭01, ('운송사01', '구간01', '트럭02'): x_운송사01_구간01_트럭02, ('운송사02', '구간01', '트럭02'): x_운송사02_구간01_트럭02, ('운송사03', '구간01', '트럭02'): x_운송사03_구간01_트럭02, ('운송사01', '구간02', '트럭01'): x_운송사01_구간02_트럭01, ('운송사02', '구간02', '트럭01'): x_운송사02_구간02_트럭01, ('운송사03', '구간02', '트럭01'): x_운송사03_구간02_트럭01, ('운송사01', '구간02', '트럭02'): x_운송사01_구간02_트럭02, ('운송사02', '구간02', '트럭02'): x_운송사02_구간02_트럭02, ('운송사03', '구간02', '트럭02'): x_운송사03_구간02_트럭02}


In [ ]:
    print(prob)

Simple_Optimization:
MINIMIZE
None
VARIABLES



# 도표 5-34. 목적식과 결정변수가 추가된 prob 객체

In [ ]:
    prob += lpSum([
        cost_dict[(i, r, t)] * x_vars[(i, r, t)]
        for (i, r, t) in cost_dict
    ]), "Minimize_TotalCost"
    print(prob)

Simple_Optimization:
MINIMIZE
100*x_운송사01_구간01_트럭01 + 100*x_운송사01_구간01_트럭02 + 100*x_운송사01_구간02_트럭01 + 700*x_운송사01_구간02_트럭02 + 150*x_운송사02_구간01_트럭01 + 150*x_운송사02_구간01_트럭02 + 150*x_운송사02_구간02_트럭01 + 350*x_운송사02_구간02_트럭02 + 200*x_운송사03_구간01_트럭01 + 200*x_운송사03_구간01_트럭02 + 200*x_운송사03_구간02_트럭01 + 400*x_운송사03_구간02_트럭02 + 0.0
VARIABLES
0 <= x_운송사01_구간01_트럭01 Integer
0 <= x_운송사01_구간01_트럭02 Integer
0 <= x_운송사01_구간02_트럭01 Integer
0 <= x_운송사01_구간02_트럭02 Integer
0 <= x_운송사02_구간01_트럭01 Integer
0 <= x_운송사02_구간01_트럭02 Integer
0 <= x_운송사02_구간02_트럭01 Integer
0 <= x_운송사02_구간02_트럭02 Integer
0 <= x_운송사03_구간01_트럭01 Integer
0 <= x_운송사03_구간01_트럭02 Integer
0 <= x_운송사03_구간02_트럭01 Integer
0 <= x_운송사03_구간02_트럭02 Integer



# 도표 5-36. 제약조건1 필요대수 충족

In [ ]:
    # 제약조건 1: 각 (구간, 트럭유형)에 대해 수요만큼 트럭이 배정되어야 함
    for (r, t) in route_type_list:
      prob += lpSum([
            x_vars[(i, r, t)]
            for i in truckers if (i, r, t) in x_vars
      ]) == demand_dict[(r, t)], f"Demand_{r}_{t}"

In [ ]:
print(prob)

Simple_Optimization:
MINIMIZE
100*x_운송사01_구간01_트럭01 + 100*x_운송사01_구간01_트럭02 + 100*x_운송사01_구간02_트럭01 + 700*x_운송사01_구간02_트럭02 + 150*x_운송사02_구간01_트럭01 + 150*x_운송사02_구간01_트럭02 + 150*x_운송사02_구간02_트럭01 + 350*x_운송사02_구간02_트럭02 + 200*x_운송사03_구간01_트럭01 + 200*x_운송사03_구간01_트럭02 + 200*x_운송사03_구간02_트럭01 + 400*x_운송사03_구간02_트럭02 + 0.0
SUBJECT TO
Demand_구간01_트럭01: x_운송사01_구간01_트럭01 + x_운송사02_구간01_트럭01 + x_운송사03_구간01_트럭01
 = 2

Demand_구간01_트럭02: x_운송사01_구간01_트럭02 + x_운송사02_구간01_트럭02 + x_운송사03_구간01_트럭02
 = 2

Demand_구간02_트럭01: x_운송사01_구간02_트럭01 + x_운송사02_구간02_트럭01 + x_운송사03_구간02_트럭01
 = 2

Demand_구간02_트럭02: x_운송사01_구간02_트럭02 + x_운송사02_구간02_트럭02 + x_운송사03_구간02_트럭02
 = 2

VARIABLES
0 <= x_운송사01_구간01_트럭01 Integer
0 <= x_운송사01_구간01_트럭02 Integer
0 <= x_운송사01_구간02_트럭01 Integer
0 <= x_운송사01_구간02_트럭02 Integer
0 <= x_운송사02_구간01_트럭01 Integer
0 <= x_운송사02_구간01_트럭02 Integer
0 <= x_운송사02_구간02_트럭01 Integer
0 <= x_운송사02_구간02_트럭02 Integer
0 <= x_운송사03_구간01_트럭01 Integer
0 <= x_운송사03_구간01_트럭02 Integer
0 <= x_운송사03_구간02_트

# 도표 5-38. 제약조건2 보유 트럭대수 초과 금지

In [ ]:
    # 제약조건 2: 운송사별 보유 트럭 수를 초과하지 않도록
    for (i, r, t), cap in capacity_dict.items():
        prob += x_vars[(i, r, t)] <= cap, f"Capacity_{i}_{r}_{t}"

In [ ]:
print(prob)

Simple_Optimization:
MINIMIZE
100*x_운송사01_구간01_트럭01 + 100*x_운송사01_구간01_트럭02 + 100*x_운송사01_구간02_트럭01 + 700*x_운송사01_구간02_트럭02 + 150*x_운송사02_구간01_트럭01 + 150*x_운송사02_구간01_트럭02 + 150*x_운송사02_구간02_트럭01 + 350*x_운송사02_구간02_트럭02 + 200*x_운송사03_구간01_트럭01 + 200*x_운송사03_구간01_트럭02 + 200*x_운송사03_구간02_트럭01 + 400*x_운송사03_구간02_트럭02 + 0.0
SUBJECT TO
Demand_구간01_트럭01: x_운송사01_구간01_트럭01 + x_운송사02_구간01_트럭01 + x_운송사03_구간01_트럭01
 = 2

Demand_구간01_트럭02: x_운송사01_구간01_트럭02 + x_운송사02_구간01_트럭02 + x_운송사03_구간01_트럭02
 = 2

Demand_구간02_트럭01: x_운송사01_구간02_트럭01 + x_운송사02_구간02_트럭01 + x_운송사03_구간02_트럭01
 = 2

Demand_구간02_트럭02: x_운송사01_구간02_트럭02 + x_운송사02_구간02_트럭02 + x_운송사03_구간02_트럭02
 = 2

Capacity_운송사01_구간01_트럭01: x_운송사01_구간01_트럭01 <= 2

Capacity_운송사02_구간01_트럭01: x_운송사02_구간01_트럭01 <= 2

Capacity_운송사03_구간01_트럭01: x_운송사03_구간01_트럭01 <= 2

Capacity_운송사01_구간01_트럭02: x_운송사01_구간01_트럭02 <= 2

Capacity_운송사02_구간01_트럭02: x_운송사02_구간01_트럭02 <= 2

Capacity_운송사03_구간01_트럭02: x_운송사03_구간01_트럭02 <= 2

Capacity_운송사01_구간02_트럭01: x_운송사01_구간02_

# 최적화 실행

In [ ]:
 prob.solve(pulp.PULP_CBC_CMD(msg=0))  # CBC Solver를 사용하여 최적화 수행

1

# 결과 출력

In [ ]:
    print("Status:", pulp.LpStatus[prob.status])  # 최적화 상태 출력
    if prob.status not in [1, 2]:  # Optimal(1) 또는 Feasible(2)인 경우만 유효
        print("=> 유효한 해를 찾지 못했습니다.")
        # return None # optimize_simple_assignment( )내 코드가 아니므로 주석처리

    # 총 운송비 계산
    total_cost = value(prob.objective)
    print("총 운송비:", total_cost)

    # 최적화 결과(트럭 배정 현황) 정리
    result_data = []
    for (i, r, t), var in x_vars.items():
        if var.varValue > 0:  # 배정된 트럭 수가 0보다 큰 경우만 출력
            result_data.append([i, r, t, cost_dict[(i, r, t)], int(var.varValue)])

    # 결과를 데이터프레임으로 출력
    df_result = pd.DataFrame(result_data, columns=['운송사', '운송구간', '트럭유형', '운송단가', '선택된트럭수'])
    print("\n--- 배정 결과 ---")
    print(df_result.to_string(index=False))  # 결과 출력

    # 엑셀로 저장
    output_file = "/content/sample_data/단순최적화결과.xlsx"
    df_result.to_excel(output_file, index=False)

    end_time = time.time()  # 종료 시간 기록
    print("\n프로그램 수행시간(초):", round(end_time - start_time, 4))  # 수행 시간 출력

    # return prob # 최적화 모델 객체 반환 # optimize_simple_assignment( )내 코드가 아니므로 주석처리


Status: Optimal
총 운송비: 1300.0

--- 배정 결과 ---
  운송사 운송구간 트럭유형  운송단가  선택된트럭수
운송사01 구간01 트럭01   100       2
운송사01 구간01 트럭02   100       2
운송사01 구간02 트럭01   100       2
운송사02 구간02 트럭02   350       2

프로그램 수행시간(초): 0.357


# 최적화 모델 상세 출력
def display_optimization_model(prob):

In [ ]:
    print("\n=== 목적함수 ===")
    print(prob.objective)  # 목적함수 출력

    print("\n=== 제약조건 ===")
    for name, constraint in prob.constraints.items():  # 제약조건 모두 출력
        print(f"{name}: {constraint}")

    print("\n=== 결정변수 ===")
    for var in prob.variables():  # 변수명과 해 출력
        print(f"{var.name}: {var.varValue}")


=== 목적함수 ===
100*x_운송사01_구간01_트럭01 + 100*x_운송사01_구간01_트럭02 + 100*x_운송사01_구간02_트럭01 + 700*x_운송사01_구간02_트럭02 + 150*x_운송사02_구간01_트럭01 + 150*x_운송사02_구간01_트럭02 + 150*x_운송사02_구간02_트럭01 + 350*x_운송사02_구간02_트럭02 + 200*x_운송사03_구간01_트럭01 + 200*x_운송사03_구간01_트럭02 + 200*x_운송사03_구간02_트럭01 + 400*x_운송사03_구간02_트럭02

=== 제약조건 ===
Demand_구간01_트럭01: x_운송사01_구간01_트럭01 + x_운송사02_구간01_트럭01 + x_운송사03_구간01_트럭01 = 2.0
Demand_구간01_트럭02: x_운송사01_구간01_트럭02 + x_운송사02_구간01_트럭02 + x_운송사03_구간01_트럭02 = 2.0
Demand_구간02_트럭01: x_운송사01_구간02_트럭01 + x_운송사02_구간02_트럭01 + x_운송사03_구간02_트럭01 = 2.0
Demand_구간02_트럭02: x_운송사01_구간02_트럭02 + x_운송사02_구간02_트럭02 + x_운송사03_구간02_트럭02 = 2.0
Capacity_운송사01_구간01_트럭01: x_운송사01_구간01_트럭01 <= 2
Capacity_운송사02_구간01_트럭01: x_운송사02_구간01_트럭01 <= 2
Capacity_운송사03_구간01_트럭01: x_운송사03_구간01_트럭01 <= 2
Capacity_운송사01_구간01_트럭02: x_운송사01_구간01_트럭02 <= 2
Capacity_운송사02_구간01_트럭02: x_운송사02_구간01_트럭02 <= 2
Capacity_운송사03_구간01_트럭02: x_운송사03_구간01_트럭02 <= 2
Capacity_운송사01_구간02_트럭01: x_운송사01_구간02_트럭01 <= 2
Capacity_운송사02_

In [ ]:
print(len(prob.constraints))  # 총 몇 개 제약조건이 정의되어 있는지 확인

16
